In [5]:
import pandas as pd, requests, datetime, json, pandas_datareader as web
from bs4 import BeautifulSoup

start_date = '03-05-2021'

# индексы
indices = ['SBERMGTR','SBERDVTI','SBERPRPT','SBPPRP85','SBERARDF','SBRBCI10',
           'SBERGLR8','SBERPPCC','SBERAGE9','SBERLV85','SBERECCY','SBERRBTI',
           'SBERDGBI','SBERREIT']

# слева, как на сайте, справа как в итоговом файле
indices_dict = {
    'SBERMGTR':'EQIND_ SBERMGTR',
    'SBERDVTI':'EQIND_SBERDVTI',
    'SBERPRPT':'PRM_PFL_IND',
    'SBPPRP85':'EQIND_SBPPRP85',
    'SBERARDF':'EQIND_SBERARDF',
    'SBRBCI10':'EQIND_SBRBCI10',
    'SBERGLR8':'EQIND_SBERGLR8',
    'SBERPPCC':'EQIND_SBERCDX',
    'SBERAGE9':'EQIND_SBERAGE9',
    'SBERLV85':'EQIND_SBERLV85',
    'SBERECCY':'EQIND_SBERECCY',
    'SBERRBTI':'EQIND_SBERRBTI',
    'SBERDGBI':'EQIND_SBERDGBI',
    'SBERREIT':'EQIND_SBERREIT'
}

In [6]:

def get_date(date): # функция преобразования даты
    date = datetime.datetime.strptime(date, '%d-%m-%Y').date()
    return date

def get_indices_from_site(website): # получить индексы
    soup = BeautifulSoup(requests.get(website).text, 'html.parser')
    header = soup.find('div', class_='header-container').find_all('option')
    indices = [option.get('value') for option in header]
    return indices

def get_currency(index): # функция для получения валюты
    page = requests.get(f'https://indices.sberbank-cib.com/?indexId={index}')
    soup = BeautifulSoup(page.text, 'html.parser')
    currency = soup.find('div', class_='index-specification-value', id='index-spec-currency').text
    return currency

def get_date_from_milliseconds(milliseconds):
    target_date = datetime.datetime(1970, 1, 1) + datetime.timedelta(0, 0, 0, milliseconds)
    return target_date.date()

def get_values(index): # функция для получения значения
    params = {'p_p_id': 'cibfiindexportlet_WAR_cibfiindexportlet_INSTANCE_dGHP3mUXPsCb', 'p_p_lifecycle': 2, 'p_p_resource_id': 'getChartData', 'extId': index}
    response = requests.get('https://indices.sberbank-cib.com/', params=params).json()
    data_values = response['chartData']['values']['data']
    for x in range(len(data_values)):
        data_values[x][0] = get_date_from_milliseconds(data_values[x][0])
    return data_values

def get_value(values, date):
    for x in values:
        if date in x:
            value = x[1]
    return value

def get_string(date, code, currency, index_type, value):
    value = str(value)
    date = str(date)
    getted_string = date +';'+ code +';'+ currency +';'+'%'+';'+ index_type +';'+ date +';advisory;'+ value +';'+ value
    return getted_string

In [7]:
data_sberbank = []
date = get_date(start_date)

# indices = get_indices_from_site('https://indices.sberbank-cib.com/') # чтобы получить все индексы с сайта

for index in indices:
    index_values = get_values(index)
    date_style = datetime.datetime.strftime(date, '%d/%m/%Y')
    try: index_value = get_value(index_values, date)
    except: index_value = 0
    try: currency = get_currency(index)
    except: currency = 'null'
    index_type = 'b'
    data_sberbank.append([date_style, date_style, indices_dict.get(index), currency, index_type, index_value, index_value, get_string(date_style, f'EQIND_{index}', currency, index_type, index_value)])

data_sberbank = pd.DataFrame(data_sberbank, columns=['start', 'end', 'code', 'currency', 'index_type', 'value_01', 'value_02', 'string'])

In [8]:
data_sberbank

,start,end,code,currency,index_type,value_01,value_02,string
0,03/05/2021,03/05/2021,EQIND_ SBERMGTR,USD,b,119.15,119.15,03/05/2021;EQIND_SBERMGTR;USD;%;b;03/05/2021;a...
1,03/05/2021,03/05/2021,EQIND_SBERDVTI,USD,b,102.15,102.15,03/05/2021;EQIND_SBERDVTI;USD;%;b;03/05/2021;a...
2,03/05/2021,03/05/2021,PRM_PFL_IND,USD,b,135.04,135.04,03/05/2021;EQIND_SBERPRPT;USD;%;b;03/05/2021;a...
3,03/05/2021,03/05/2021,EQIND_SBPPRP85,USD,b,94.90,94.90,03/05/2021;EQIND_SBPPRP85;USD;%;b;03/05/2021;a...
4,03/05/2021,03/05/2021,EQIND_SBERARDF,USD,b,114.04,114.04,03/05/2021;EQIND_SBERARDF;USD;%;b;03/05/2021;a...
5,03/05/2021,03/05/2021,EQIND_SBRBCI10,RUR,b,0.00,0.00,03/05/2021;EQIND_SBRBCI10;RUR;%;b;03/05/2021;a...
6,03/05/2021,03/05/2021,EQIND_SBERGLR8,USD,b,111.38,111.38,03/05/2021;EQIND_SBERGLR8;USD;%;b;03/05/2021;a...
7,03/05/2021,03/05/2021,EQIND_SBERCDX,USD,b,0.00,0.00,03/05/2021;EQIND_SBERPPCC;USD;%;b;03/05/2021;a...
8,03/05/2021,03/05/2021,EQIND_SBERAGE9,USD,b,107.02,107.02,03/05/2021;EQIND_SBERAGE9;USD;%;b;03/05/2021;a...
9,03/05/2021,03/05/2021,EQIND_SBERLV85,USD,b,113.62,113.62,03/05/2021;EQIND_SBERLV85;USD;%;b;03/05/2021;a...
